<a href="https://colab.research.google.com/github/lima-breno/natural_language_processing/blob/main/Classificador_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Carregando o embedding e os dados

In [ ]:
!pip install unidecode
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
import gensim
import pandas as pd
from nltk.corpus import stopwords
import string
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import re

ModuleNotFoundError: No module named 'gensim'

In [ ]:
#opção 1 -> montar o drive no colab e acessar o arquivo de embedding do drive
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
%%time
path='drive/MyDrive/aulas/Processamento de Linguagem Natural/ptwiki_20180420_100d.txt.bz2'
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)

## Tratamento dos dados

1. Iremos tornar a nossa variável alvo como binaria
2. Iremos criar um pré-processamento
3. Iremos criar a representação textual

In [ ]:
df = pd.read_csv('drive/MyDrive/aulas/Processamento de Linguagem Natural/imdb-reviews-pt-br.csv')

In [ ]:
df.head()

In [ ]:
target = df['sentiment'].replace(['neg','pos'],[0,1])

In [ ]:
def pre_processamento_texto_return_str(corpus, portugues_stops):
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus) # extração palavras -> palavras
    corpus_alt = [t.lower() for t in corpus_alt] #passando para minusculo
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops] #remoção dos tokens que são SW
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation] # o mesmo para pontuação
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt] #remoção de numeros
    corpus_alt_str = ' '.join(corpus_alt)
    return corpus_alt_str

In [ ]:
def pre_processamento_texto_return_token(corpus, portugues_stops):
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    corpus_alt = [t.lower() for t in corpus_alt]
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt]

    return corpus_alt

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
portugues_stops = stopwords.words('portuguese')

In [ ]:
from tqdm import tqdm
tqdm.pandas()

df["text_pt_sem_stopwords"] = df["text_pt"]\
      .progress_apply(lambda x: pre_processamento_texto_return_str(x, portugues_stops))

In [ ]:
df.head()

## Bag-of-word

In [ ]:
vect_bag = CountVectorizer()
X_bag = vect_bag.fit_transform(df['text_pt_sem_stopwords'])
vocabulario = vect_bag.get_feature_names_out()

In [ ]:
print("Vocabulario",len(vocabulario))
print("Features",X_bag.shape)
print("Target",target.shape)

## Embedding

In [ ]:
df["text_pt_sem_stopwords_token"] = df["text_pt"]\
.progress_apply(lambda x: pre_processamento_texto_return_token(x, portugues_stops))

In [ ]:
import numpy as np
def calcula_embedding_frase(tokens):
    return np.mean([word_vectors[t] for t in tokens if t in word_vectors.key_to_index.keys()], axis=0)

In [ ]:
X_embedding = df["text_pt_sem_stopwords_token"].progress_apply(lambda x: calcula_embedding_frase(x))

In [ ]:
X_embedding

### Treinamento

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = \
    train_test_split(X_bag, target, random_state=123)

In [ ]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_bow,y_train_bow)

In [ ]:
y_pred = modelo_bow.predict(X_test_bow)

In [ ]:
print(classification_report(y_test_bow, y_pred))

### Embedding

In [ ]:
X_train_embedding, X_test_embedding, y_train_embedding, y_test_embedding = \
train_test_split(X_embedding.values, target,random_state=123)

In [ ]:
X_train_embedding = pd.DataFrame([x for x in X_train_embedding])
X_test_embedding = pd.DataFrame([x for x in X_test_embedding])

In [ ]:
X_train_embedding.head()

In [ ]:
modelo_embedding = LogisticRegression()
modelo_embedding.fit(X_train_embedding,y_train_embedding)

In [ ]:
y_pred = modelo_embedding.predict(X_test_embedding)

In [ ]:
print(classification_report(y_test_embedding, y_pred))

# Análise de sentimentos

Quando o objetivo é realizar análise de sentimentos podemos treinar o nosso proprio modelo ou utilizar ferramentas já feitas. Exemplo: Vader.

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

In [ ]:
analyzer.polarity_scores(texto_neg)

In [ ]:
analyzer.polarity_scores(df.loc[2, "text_en"])

In [ ]:
analyzer.polarity_scores(texto_pos)